In [7]:
install.packages("SunterSampling")

Installing package into 'C:/Users/pleal/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)



Warning message:
"package 'SunterSampling' is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages"


In [ ]:
# Requiere el paquete openxlsx para exportar a Excel
if (!requireNamespace("openxlsx", quietly = TRUE)) {
  install.packages("openxlsx")
}
library(openxlsx)

# Función interna para calcular pi_kl usando el método de Sunter (sin inclusiones forzadas)
sunterpi2_core <- function(x, n) {
  N <- length(x)
  if (x[1] * n / sum(x) >= 1) stop("Error interno: hay unidades con pi_k >= 1 en sunterpi2_core.")
  
  t <- rev(cumsum(rev(x)))
  xbar <- t / (N:1)
  kk0 <- n * x / t
  k0 <- which(kk0 >= 1, arr.ind = FALSE)[1]
  if (is.na(k0)) k0 <- N + 1
  kstar <- min(k0, N - n + 1)
  
  # Calcular los valores g_k
  g <- numeric(kstar)
  g[1] <- 1 / t[2]
  for (i in 2:kstar) {
    g[i] <- prod(1 - x[1:(i-1)] / t[2:i]) / t[i+1]
  }
  
  # Inicializar la matriz de pi_kl
  piij <- matrix(0, N, N)
  
  # Calcular pi_kl según las condiciones de k y l respecto a k*
  for (k in 1:(N-1)) {
    for (l in (k+1):N) {
      if (k < l & l < kstar) {
        piij[k, l] <- n * (n - 1) / t[1] * g[k] * x[k] * x[l]
      } else if (k < kstar & kstar <= l) {
        piij[k, l] <- n * (n - 1) / t[1] * g[k] * x[k] * xbar[kstar]
      } else if (kstar <= k & k < l) {
        piij[k, l] <- n * (n - 1) / t[1] * g[kstar - 1] * (t[kstar] - x[kstar - 1]) / 
          (t[kstar] - xbar[kstar]) * xbar[kstar]^2
      }
      piij[l, k] <- piij[k, l]  # Simetría
    }
  }
  
  # Ajustar la diagonal con las probabilidades de primer orden
  diag(piij) <- apply(piij, 1, sum) / (n - 1)
  
  return(piij)
}

# Función para manejar inclusiones forzadas de manera recursiva
handle_forced_inclusions <- function(x, n) {
  N <- length(x)
  total_x <- sum(x)
  pi_initial <- n * x / total_x
  forced <- which(pi_initial >= 1)
  n_forced <- length(forced)
  
  if (n_forced == 0) {
    return(list(forced = integer(0), x_remaining = x, n_remaining = n))
  }
  
  if (n_forced >= n) {
    stop("El número de inclusiones forzadas excede el tamaño de la muestra.")
  }
  
  # Unidades restantes
  x_remaining <- x[-forced]
  n_remaining <- n - n_forced
  
  # Verificar recursivamente si hay más inclusiones forzadas en las restantes
  further_forced <- handle_forced_inclusions(x_remaining, n_remaining)
  
  # Combinar las inclusiones forzadas
  forced <- c(forced, further_forced$forced + n_forced)
  x_remaining <- further_forced$x_remaining
  n_remaining <- further_forced$n_remaining
  
  return(list(forced = forced, x_remaining = x_remaining, n_remaining = n_remaining))
}

# Función principal para calcular pi_kl con manejo de inclusiones forzadas y exportar a Excel
sunterpi2_with_forced <- function(x, n, excel_file = "pi_kl_matrix.xlsx") {
  # Ordenar las medidas de tamaño en orden decreciente
  xo <- sort(x, decreasing = TRUE)
  N <- length(xo)
  
  # Manejar inclusiones forzadas de manera recursiva
  forced_info <- handle_forced_inclusions(xo, n)
  forced <- forced_info$forced
  x_remaining <- forced_info$x_remaining
  n_remaining <- forced_info$n_remaining
  n_forced <- length(forced)
  
  if (n_forced > 0) {
    cat("Unidades con inclusión forzada:", n_forced, "\n")
    
    if (n_remaining <= 0) {
      cat("No quedan unidades para muestrear después de inclusiones forzadas.\n")
      piij <- matrix(0, N, N)
      diag(piij) <- c(rep(1, n_forced), rep(0, N - n_forced))
      
      # Exportar a Excel
      write.xlsx(as.data.frame(piij), file = excel_file, rowNames = TRUE, colNames = TRUE)
      return(piij)
    }
    
    if (length(x_remaining) < n_remaining) stop("No hay suficientes unidades restantes para el tamaño de muestra ajustado.")
    
    # Calcular pi_kl para las unidades restantes usando el método de Sunter
    piij_remaining <- sunterpi2_core(x_remaining, n_remaining)
    N_remaining <- length(x_remaining)
    
    # Construir la matriz completa de pi_kl
    piij <- matrix(0, N, N)
    
    # Llenar las probabilidades para las unidades con inclusión forzada
    for (k in forced) {
      piij[k, k] <- 1  # pi_k = 1 para unidades forzadas
      for (l in 1:N) {
        if (l %in% forced && l != k) {
          piij[k, l] <- 1  # Ambas unidades forzadas: pi_kl = 1
        } else if (!(l %in% forced)) {
          # Ajustar índice para unidades restantes
          l_adj <- match(l, which(!(1:N %in% forced)))
          if (l_adj > N_remaining) stop("Índice fuera de límites en piij_remaining")
          piij[k, l] <- piij_remaining[l_adj, l_adj]  # pi_kl = pi_l de las restantes
        }
        piij[l, k] <- piij[k, l]  # Simetría
      }
    }
    
    # Llenar las probabilidades para las unidades restantes
    remaining_indices <- which(!(1:N %in% forced))
    for (k in remaining_indices) {
      k_adj <- match(k, remaining_indices)
      for (l in remaining_indices[remaining_indices > k]) {
        l_adj <- match(l, remaining_indices)
        piij[k, l] <- piij_remaining[k_adj, l_adj]
        piij[l, k] <- piij[k, l]  # Simetría
      }
      piij[k, k] <- piij_remaining[k_adj, k_adj]
    }
    
  } else {
    # Si no hay inclusiones forzadas, usar directamente el método de Sunter
    piij <- sunterpi2_core(xo, n)
  }
  
  # Reordenar la matriz según el orden original de x
  ord <- order(x, decreasing = TRUE)
  piij <- piij[ord, ord]
  
  # Exportar la matriz a Excel
  write.xlsx(as.data.frame(piij), file = excel_file, rowNames = TRUE, colNames = TRUE)
  cat("Matriz pi_kl exportada a:", excel_file, "\n")
  
  return(piij)
}

# Ejemplo de uso
x <- c(312979, 102382, 67324, 52367, 32009, 29623, 28032, 27199, 26582, 23738, 
       23014, 19748, 19738, 16547, 16002, 15898, 14473, 13423, 12788, 11665, 
       10268, 10247, 10163, 10096, 9876, 8811, 8479, 8426, 8174, 7798, 
       7308, 6468, 5781, 5169, 4330, 3596, 3321)
n <- 8
pi_kl <- sunterpi2_with_forced(x, n, excel_file = "pi_kl_matrix.xlsx")

ERROR: Error in read.xlsx.default(excel_file, sheet = 1, startRow = 3, colNames = TRUE): File does not exist.


In [1]:
# Requiere el paquete openxlsx para exportar a Excel
if (!requireNamespace("openxlsx", quietly = TRUE)) {
  install.packages("openxlsx")
}
library(openxlsx)

# --- No se necesitan cambios en las funciones internas ---

# Función interna para calcular pi_kl usando el método de Sunter (sin inclusiones forzadas)
sunterpi2_core <- function(x, n) {
  N <- length(x)
  if (x[1] * n / sum(x) >= 1) stop("Error interno: hay unidades con pi_k >= 1 en sunterpi2_core.")
  
  t <- rev(cumsum(rev(x)))
  xbar <- t / (N:1)
  kk0 <- n * x / t
  k0 <- which(kk0 >= 1, arr.ind = FALSE)[1]
  if (is.na(k0)) k0 <- N + 1
  kstar <- min(k0, N - n + 1)
  
  g <- numeric(kstar)
  g[1] <- 1 / t[2]
  for (i in 2:kstar) {
    g[i] <- prod(1 - x[1:(i-1)] / t[2:i]) / t[i+1]
  }
  
  piij <- matrix(0, N, N)
  
  for (k in 1:(N-1)) {
    for (l in (k+1):N) {
      if (k < l & l < kstar) {
        piij[k, l] <- n * (n - 1) / t[1] * g[k] * x[k] * x[l]
      } else if (k < kstar & kstar <= l) {
        piij[k, l] <- n * (n - 1) / t[1] * g[k] * x[k] * xbar[kstar]
      } else if (kstar <= k & k < l) {
        piij[k, l] <- n * (n - 1) / t[1] * g[kstar - 1] * (t[kstar] - x[kstar - 1]) / 
          (t[kstar] - xbar[kstar]) * xbar[kstar]^2
      }
      piij[l, k] <- piij[k, l]
    }
  }
  
  diag(piij) <- apply(piij, 1, sum) / (n - 1)
  
  return(piij)
}

# Función para manejar inclusiones forzadas de manera recursiva
handle_forced_inclusions <- function(x, n) {
  N <- length(x)
  total_x <- sum(x)
  pi_initial <- n * x / total_x
  forced <- which(pi_initial >= 1)
  n_forced <- length(forced)
  
  if (n_forced == 0) {
    return(list(forced = integer(0), x_remaining = x, n_remaining = n))
  }
  
  if (n_forced >= n) {
    stop("El número de inclusiones forzadas excede el tamaño de la muestra.")
  }
  
  x_remaining <- x[-forced]
  n_remaining <- n - n_forced
  
  further_forced <- handle_forced_inclusions(x_remaining, n_remaining)
  
  forced <- c(forced, further_forced$forced + n_forced)
  x_remaining <- further_forced$x_remaining
  n_remaining <- further_forced$n_remaining
  
  return(list(forced = forced, x_remaining = x_remaining, n_remaining = n_remaining))
}


# --- Función principal modificada ---

#' @title Calcula probabilidades de inclusión de primer y segundo orden
#' @description Utiliza el método de Sunter para calcular las probabilidades de
#' inclusión de segundo orden (pi_kl), y a partir de ellas, las de primer
#' orden (pi_k) y la matriz de covarianza (Delta_kl).
#' @param x Vector numérico con las medidas de tamaño de la población.
#' @param n Tamaño de la muestra.
#' @param excel_file Nombre del archivo Excel para exportar los resultados.
#' @return Una lista con tres elementos:
#'         - `pi_k`: Vector de probabilidades de inclusión de primer orden.
#'         - `pi_kl`: Matriz de probabilidades de inclusión de segundo orden.
#'         - `delta_kl`: Matriz de covarianza de los indicadores de inclusión.
#'
sunterpi2_with_forced <- function(x, n, excel_file = "resultados_sunter.xlsx") {
  # Ordenar las medidas de tamaño en orden decreciente
  xo <- sort(x, decreasing = TRUE)
  N <- length(xo)
  
  # Manejar inclusiones forzadas
  forced_info <- handle_forced_inclusions(xo, n)
  forced <- forced_info$forced
  x_remaining <- forced_info$x_remaining
  n_remaining <- forced_info$n_remaining
  n_forced <- length(forced)
  
  if (n_forced > 0) {
    cat("Unidades con inclusión forzada:", n_forced, "\n")
    
    if (n_remaining <= 0) {
      cat("No quedan unidades para muestrear después de inclusiones forzadas.\n")
      piij <- matrix(0, N, N)
      diag(piij)[1:n_forced] <- 1
      
    } else {
      if (length(x_remaining) < n_remaining) stop("No hay suficientes unidades restantes para el tamaño de muestra ajustado.")
      
      # Calcular pi_kl para las unidades restantes
      piij_remaining <- sunterpi2_core(x_remaining, n_remaining)
      N_remaining <- length(x_remaining)
      
      # Construir la matriz completa de pi_kl
      piij <- matrix(0, N, N)
      remaining_indices <- which(!(1:N %in% forced))
      
      # Llenar probabilidades para unidades forzadas
      for (k in forced) {
        piij[k, k] <- 1 # pi_k = 1
        for (l in 1:N) {
          if (l %in% forced && l != k) {
            piij[k, l] <- 1
          } else if (!(l %in% forced)) {
            l_adj <- match(l, remaining_indices)
            piij[k, l] <- diag(piij_remaining)[l_adj] # pi_kl = pi_l para las restantes
          }
          piij[l, k] <- piij[k, l]
        }
      }
      
      # Llenar probabilidades para unidades restantes
      for (i in 1:N_remaining) {
        for (j in 1:N_remaining) {
          k <- remaining_indices[i]
          l <- remaining_indices[j]
          piij[k, l] <- piij_remaining[i, j]
        }
      }
    }
    
  } else {
    # Si no hay inclusiones forzadas, usar directamente el método de Sunter
    piij <- sunterpi2_core(xo, n)
  }
  
  # Reordenar la matriz según el orden original de x
  ord <- order(order(x, decreasing = TRUE))
  piij <- piij[ord, ord]
  
  # --- NUEVOS CÁLCULOS Y SALIDAS ---
  
  # 1. Extraer el vector de probabilidades de primer orden (pi_k) de la diagonal
  pi_k <- diag(piij)
  
  # 2. Imprimir el vector pi_k en la consola
  cat("\n--- Vector de probabilidades de primer orden (pi_k) ---\n")
  print(pi_k)
  
  # 3. Calcular la matriz de covarianza (Delta_kl = pi_kl - pi_k * pi_l)
  # El producto pi_k * pi_l se calcula con el producto exterior de vectores
  delta_kl <- piij - (pi_k %o% pi_k)
  
  # 4. Exportar ambas matrices a un archivo Excel con dos hojas
  wb <- createWorkbook()
  
  # Hoja para pi_kl
  addWorksheet(wb, "pi_kl")
  writeData(wb, "pi_kl", as.data.frame(piij), rowNames = TRUE, colNames = TRUE)
  
  # Hoja para delta_kl
  addWorksheet(wb, "delta_kl")
  writeData(wb, "delta_kl", as.data.frame(delta_kl), rowNames = TRUE, colNames = TRUE)
  
  saveWorkbook(wb, excel_file, overwrite = TRUE)
  cat("\nMatrices pi_kl y delta_kl exportadas a:", excel_file, "\n")
  
  # 5. Devolver una lista con todos los resultados
  return(list(pi_k = pi_k, pi_kl = piij, delta_kl = delta_kl))
}

# --- Ejemplo de uso actualizado ---

x <- c(312979, 102382, 67324, 52367, 32009, 29623, 28032, 27199, 26582, 23738, 
       23014, 19748, 19738, 16547, 16002, 15898, 14473, 13423, 12788, 11665, 
       10268, 10247, 10163, 10096, 9876, 8811, 8479, 8426, 8174, 7798, 
       7308, 6468, 5781, 5169, 4330, 3596, 3321)
n <- 8

# Ejecutar la función y guardar los resultados en una lista
resultados <- sunterpi2_with_forced(x, n, excel_file = "resultados_sunter.xlsx")

# Ahora puedes acceder a cada resultado desde la lista 'resultados'
# Por ejemplo, para ver las primeras 6 filas de cada elemento:
cat("\n--- Primeros 6 elementos de pi_k ---\n")
print(head(resultados$pi_k))

cat("\n--- Esquina superior izquierda de la matriz pi_kl ---\n")
print(head(resultados$pi_kl[, 1:6]))

cat("\n--- Esquina superior izquierda de la matriz delta_kl ---\n")
print(head(resultados$delta_kl[, 1:6]))

Warning message:
"package 'openxlsx' was built under R version 4.4.3"


Unidades con inclusión forzada: 2 

--- Vector de probabilidades de primer orden (pi_k) ---
 [1] 1.00000000 1.00000000 0.69828395 0.54315008 0.33199708 0.30724950
 [7] 0.29074767 0.28210780 0.27570828 0.24621033 0.23870101 0.20482609
[13] 0.20472237 0.17162534 0.16597261 0.16489392 0.15011383 0.13922324
[19] 0.13263703 0.12098928 0.10649961 0.10628180 0.10541055 0.10471563
[25] 0.10243379 0.09138762 0.08794412 0.08739440 0.08478066 0.05674905
[31] 0.05674905 0.05674905 0.05674905 0.05674905 0.05674905 0.05674905
[37] 0.05674905

Matrices pi_kl y delta_kl exportadas a: resultados_sunter.xlsx 

--- Primeros 6 elementos de pi_k ---
[1] 1.0000000 1.0000000 0.6982840 0.5431501 0.3319971 0.3072495

--- Esquina superior izquierda de la matriz pi_kl ---
          [,1]      [,2]      [,3]      [,4]       [,5]       [,6]
[1,] 1.0000000 1.0000000 0.6982840 0.5431501 0.33199708 0.30724950
[2,] 1.0000000 1.0000000 0.6982840 0.5431501 0.33199708 0.30724950
[3,] 0.6982840 0.6982840 0.6982840 0.357688